In [ ]:
# Install requirements (if needed)
!pip install torch torchvision pandas matplotlib pillow

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from torchvision.models import ResNet50_Weights
import pandas as pd
from PIL import Image
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import time
from torchvision import models
from torch.utils.data import DataLoader
import numpy as np
from typing import Dict, Tuple, List
import datetime


# Configuration
class Config:
    root_dir = 'D:/celeba/img_align_celeba/img_align_celeba'
    csv_path = 'D:/celeba/list_attr_celeba.csv'

    num_classes = 40
    batch_size = 64
    lr = 1e-3
    patience = 3
    train_ratio = 0.8
    image_size = 224
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    pretrained_weights = models.ResNet50_Weights.IMAGENET1K_V2
    max_epochs = 10
    grad_clip = 1.0
    grad_accum_steps = 2
    log_interval = 10
    early_stop_patience = 5
    attribute_names = [f"Attr_{i}" for i in range(40)]  # Replace with actual names

# Dataset class
class CelebADataset(Dataset):
    def __init__(self, root_dir, csv_path, transform=None):
        self.df = pd.read_csv(csv_path)
        self.df.iloc[:, 1:] = self.df.iloc[:, 1:].replace(-1, 0)
        self.root_dir = root_dir
        self.transform = transform or transforms.Compose([
            transforms.Resize(256),
            transforms.RandomCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(Config.mean, Config.std)
        ])
        self.image_names = self.df['image_id']
        self.labels = self.df.drop('image_id', axis=1).values.astype('float32')

    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_names[idx])
        try:
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
        except Exception as e:
            print(f"Error loading {img_path}: {str(e)}")
            # Return zero image and corresponding label
            image = torch.zeros(3, 224, 224)
            return image, self.labels[idx]

        return image, self.labels[idx]  # Fixed the label reference here

# Initialize device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load dataset
dataset = CelebADataset(Config.root_dir, Config.csv_path)
train_size = int(len(dataset) * Config.train_ratio)
train_set, val_set = random_split(dataset, [train_size, len(dataset)-train_size])

    # Data loader configuration
train_loader = DataLoader(
        train_set,
        batch_size=Config.batch_size,
        shuffle=True,
        num_workers=0,  # Critical for Windows/Jupyter stability
        pin_memory=True,
        persistent_workers=False
    )

val_loader = DataLoader(
        val_set,
        batch_size=Config.batch_size,
        num_workers=0,
        pin_memory=True
    )

Using device: cuda


In [ ]:
class AttributeClassifier:
    def __init__(self, config):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self._build_model()
        self.optimizer = self._configure_optimizer()
        self.criterion = nn.BCEWithLogitsLoss()
        self._init_history()

    def _init_history(self):
        self.history = {
            'train_loss': [], 'val_loss': [],
            'train_acc': [], 'val_acc': [],
            'train_f1': [], 'val_f1': [],
            'train_precision': [], 'val_precision': [],
            'train_recall': [], 'val_recall': [],
            'train_attr_metrics': {i: {'acc': [], 'f1': [], 'precision': [], 'recall': []}
                                 for i in range(self.config.num_classes)},
            'val_attr_metrics': {i: {'acc': [], 'f1': [], 'precision': [], 'recall': []}
                               for i in range(self.config.num_classes)},
            'learning_rates': []
        }

    def _build_model(self) -> nn.Module:
        # Load pretrained ResNet50
        model = models.resnet50(weights=self.config.pretrained_weights)

        # Freeze all layers except final blocks
        for param in model.parameters():
            param.requires_grad = False

        # Unfreeze layer4 and classifier
        for param in model.layer3.parameters():
            param.requires_grad = True

        for param in model.layer4.parameters():
            param.requires_grad = True

        # Replace final fully connected layer
        model.fc = nn.Sequential(
            nn.Linear(model.fc.in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, self.config.num_classes)
        )

        # Initialize classifier weights
        for layer in model.fc:
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_normal_(layer.weight)
                nn.init.constant_(layer.bias, 0)

        return model.to(self.device)

    def _configure_optimizer(self) -> optim.Optimizer:
        # Only optimize unfrozen parameters
        return optim.AdamW(
            filter(lambda p: p.requires_grad, self.model.parameters()),
            lr=self.config.lr,
            weight_decay=0.01
        )

    def train(self, train_loader: DataLoader, val_loader: DataLoader) -> Dict:
        scaler = torch.cuda.amp.GradScaler()
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, 'max', patience=3, factor=0.5, verbose=True
        )
        best_f1 = 0
        epochs_no_improve = 0
        start_train_time = time.time()  # Track total training start time

        for epoch in range(self.config.max_epochs):
            epoch_start_time = time.time()
            epoch_start_str = time.strftime("%H:%M:%S")  # Human-readable start time

            # Training Phase
            self.model.train()
            epoch_loss = 0
            for batch_idx, (inputs, labels) in enumerate(train_loader):
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                # Forward pass with mixed precision
                with torch.cuda.amp.autocast():
                    outputs = self.model(inputs)
                    loss = self.criterion(outputs, labels) / self.config.grad_accum_steps

                # Backward pass and gradient accumulation
                scaler.scale(loss).backward()

                if (batch_idx + 1) % self.config.grad_accum_steps == 0:
                    scaler.unscale_(self.optimizer)
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.config.grad_clip)
                    scaler.step(self.optimizer)
                    scaler.update()
                    self.optimizer.zero_grad()

                epoch_loss += loss.item() * self.config.grad_accum_steps

                # Batch progress logging
                if batch_idx % self.config.log_interval == 0:
                    print(f'\rEpoch {epoch+1} Batch {batch_idx}/{len(train_loader)} '
                          f'Loss: {loss.item():.4f}', end='')

            # Calculate training loss and store
            train_loss = epoch_loss / len(train_loader)
            self.history['train_loss'].append(train_loss)

            # Validation Phase
            train_metrics = self._compute_epoch_metrics(train_loader, 'train')
            val_metrics = self._compute_epoch_metrics(val_loader, 'val')

            # Learning rate scheduling
            current_lr = self.optimizer.param_groups[0]['lr']
            self.history['learning_rates'].append(current_lr)
            scheduler.step(val_metrics['f1'])

            # Calculate time metrics
            epoch_duration = time.time() - epoch_start_time
            total_duration = time.time() - start_train_time

            # Print epoch summary with new details
            self._print_epoch_summary(
                epoch=epoch,
                train_metrics=train_metrics,
                val_metrics=val_metrics,
                epoch_start_str=epoch_start_str,
                epoch_duration=epoch_duration,
                total_duration=total_duration
            )

            # Early stopping check
            if val_metrics['f1'] > best_f1:
                best_f1 = val_metrics['f1']
                epochs_no_improve = 0
                torch.save(self.model.state_dict(), 'best_model.pth')
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= self.config.early_stop_patience:
                    print(f"\nEarly stopping triggered after {epoch+1} epochs!")
                    break

        # Final model evaluation
        self.model.load_state_dict(torch.load('best_model.pth'))
        self._print_final_metrics(val_loader)
        return self.history

    def _print_epoch_summary(self, epoch: int, train_metrics: Dict,
                           val_metrics: Dict, epoch_start_str: str,
                           epoch_duration: float, total_duration: float):
        """Enhanced epoch summary with time metrics and learning rate"""
        # Format time durations
        epoch_time = str(datetime.timedelta(seconds=int(epoch_duration)))
        total_time = str(datetime.timedelta(seconds=int(total_duration)))

        # Get current learning rate
        current_lr = self.history['learning_rates'][epoch]

        print(f"\nEpoch {epoch+1}/{self.config.max_epochs} Summary:")
        print(f"Start Time: {epoch_start_str} | Duration: {epoch_time} | Total Elapsed: {total_time}")
        print(f"Learning Rate: {current_lr:.2e}")
        print(f"Train Loss: {self.history['train_loss'][-1]:.4f} | Val Loss: {val_metrics['loss']:.4f}")
        print(f"Train Acc: {train_metrics['accuracy']:.4f} | Val Acc: {val_metrics['accuracy']:.4f}")
        print(f"Train F1: {train_metrics['f1']:.4f} | Val F1: {val_metrics['f1']:.4f}")
        print(f"Train Precision: {train_metrics['precision']:.4f} | Val Precision: {val_metrics['precision']:.4f}")
        print(f"Train Recall: {train_metrics['recall']:.4f} | Val Recall: {val_metrics['recall']:.4f}")

    def _compute_epoch_metrics(self, data_loader: DataLoader, phase: str) -> Dict:
        """Compute metrics for an entire epoch."""
        self.model.eval() if phase == 'val' else self.model.train()
        metrics = {
            'loss': 0,
            'accuracy': 0,
            'f1': 0,
            'precision': 0,
            'recall': 0,
            'per_attribute': {
                i: {'tp': 0, 'fp': 0, 'fn': 0, 'tn': 0}
                for i in range(self.config.num_classes)
            }
        }

        with torch.no_grad():
            for inputs, labels in data_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)

                # Calculate loss
                if phase == 'val':
                    metrics['loss'] += self.criterion(outputs, labels).item()

                # Calculate predictions
                preds = (torch.sigmoid(outputs) > 0.5).float()

                # Per-attribute metrics
                for attr in range(self.config.num_classes):
                    tp = ((preds[:, attr] == 1) & (labels[:, attr] == 1)).sum().item()
                    fp = ((preds[:, attr] == 1) & (labels[:, attr] == 0)).sum().item()
                    fn = ((preds[:, attr] == 0) & (labels[:, attr] == 1)).sum().item()
                    tn = ((preds[:, attr] == 0) & (labels[:, attr] == 0)).sum().item()

                    metrics['per_attribute'][attr]['tp'] += tp
                    metrics['per_attribute'][attr]['fp'] += fp
                    metrics['per_attribute'][attr]['fn'] += fn
                    metrics['per_attribute'][attr]['tn'] += tn

        # Aggregate metrics
        return self._aggregate_metrics(metrics, len(data_loader), phase)

    def _aggregate_metrics(self, metrics: Dict, num_batches: int, phase: str) -> Dict:
        if phase == 'val':
            metrics['loss'] /= num_batches

        attr_acc, attr_f1 = [], []
        attr_precision, attr_recall = [], []

        for attr in range(self.config.num_classes):
            tp = metrics['per_attribute'][attr]['tp']
            fp = metrics['per_attribute'][attr]['fp']
            fn = metrics['per_attribute'][attr]['fn']
            tn = metrics['per_attribute'][attr]['tn']

            acc = (tp + tn) / (tp + tn + fp + fn + 1e-9)
            precision = tp / (tp + fp + 1e-9)
            recall = tp / (tp + fn + 1e-9)
            f1 = 2 * (precision * recall) / (precision + recall + 1e-9)

            self.history[f'{phase}_attr_metrics'][attr]['acc'].append(acc)
            self.history[f'{phase}_attr_metrics'][attr]['f1'].append(f1)
            self.history[f'{phase}_attr_metrics'][attr]['precision'].append(precision)
            self.history[f'{phase}_attr_metrics'][attr]['recall'].append(recall)

            attr_acc.append(acc)
            attr_f1.append(f1)
            attr_precision.append(precision)
            attr_recall.append(recall)

        metrics.update({
            'accuracy': np.mean(attr_acc),
            'f1': np.mean(attr_f1),
            'precision': np.mean(attr_precision),
            'recall': np.mean(attr_recall)
        })

        self.history[f'{phase}_acc'].append(metrics['accuracy'])
        self.history[f'{phase}_f1'].append(metrics['f1'])
        self.history[f'{phase}_precision'].append(metrics['precision'])
        self.history[f'{phase}_recall'].append(metrics['recall'])

        if phase == 'val':
            self.history['val_loss'].append(metrics['loss'])
        else:
            self.history['train_loss'].append(metrics['loss'])

        return metrics

    def _print_epoch_summary(self, epoch: int, train_metrics: Dict, val_metrics: Dict):
        print(f"\nEpoch {epoch+1} Summary:")
        print(f"Train Loss: {self.history['train_loss'][-1]:.4f} | Val Loss: {val_metrics['loss']:.4f}")
        print(f"Train Acc: {train_metrics['accuracy']:.4f} | Val Acc: {val_metrics['accuracy']:.4f}")
        print(f"Train F1: {train_metrics['f1']:.4f} | Val F1: {val_metrics['f1']:.4f}")

    def _print_final_metrics(self, val_loader: DataLoader):
        print("\nFinal Validation Performance for All Attributes:")
        print(f"{'Attribute':<25} {'Accuracy':<8} {'F1':<8} {'Precision':<10} {'Recall':<10}")

        # Get final validation metrics
        final_metrics = {
            attr: {
                'acc': self.history['val_attr_metrics'][attr]['acc'][-1],
                'f1': self.history['val_attr_metrics'][attr]['f1'][-1],
                'precision': self.history['val_attr_metrics'][attr]['precision'][-1],
                'recall': self.history['val_attr_metrics'][attr]['recall'][-1]
            }
            for attr in range(self.config.num_classes)
        }

        for attr in range(self.config.num_classes):
            print(f"{self.config.attribute_names[attr]:<25} "
                  f"{final_metrics[attr]['acc']:.4f}    "
                  f"{final_metrics[attr]['f1']:.4f}    "
                  f"{final_metrics[attr]['precision']:.4f}      "
                  f"{final_metrics[attr]['recall']:.4f}")

# Usage Example
if __name__ == "__main__":
    config = Config()
    classifier = AttributeClassifier(config)

    history = classifier.train(train_loader, val_loader)
    classifier.generate_visualizations()

C:\Users\akash\AppData\Local\Temp\ipykernel_28732\1565594552.py:64: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
C:\Users\akash\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
C:\Users\akash\AppData\Local\Temp\ipykernel_28732\1565594552.py:83: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 1 Batch 2530/2533 Loss: 0.0964
Epoch 1/10 Summary:
Start Time: 02:36:36 | Duration: 1:00:48 | Total Elapsed: 1:00:48
Learning Rate: 1.00e-03
Train Loss: 0.0000 | Val Loss: 0.1985
Train Acc: 0.9101 | Val Acc: 0.9127
Train F1: 0.6866 | Val F1: 0.6896
Train Precision: 0.7505 | Val Precision: 0.7633
Train Recall: 0.6536 | Val Recall: 0.6528
Epoch 2 Batch 2530/2533 Loss: 0.1018
Epoch 2/10 Summary:
Start Time: 03:37:24 | Duration: 0:53:04 | Total Elapsed: 1:53:52
Learning Rate: 1.00e-03
Train Loss: 0.0000 | Val Loss: 0.1938
Train Acc: 0.9138 | Val Acc: 0.9147
Train F1: 0.6962 | Val F1: 0.6959
Train Precision: 0.7769 | Val Precision: 0.7840
Train Recall: 0.6522 | Val Recall: 0.6496
Epoch 3 Batch 2530/2533 Loss: 0.0965
Epoch 3/10 Summary:
Start Time: 04:30:29 | Duration: 0:57:07 | Total Elapsed: 2:51:00
Learning Rate: 1.00e-03
Train Loss: 0.0000 | Val Loss: 0.1900
Train Acc: 0.9158 | Val Acc: 0.9163
Train F1: 0.7043 | Val F1: 0.7024
Train Precision: 0.7837 | Val Precision: 0.7869
Train R

AttributeError: 'AttributeClassifier' object has no attribute '_print_final_metrics'